<a href="https://colab.research.google.com/github/RR77ui/Business-Intelligence/blob/main/Integracion%20de%20datos/Trabajo_Final_Hidroelectrica/Final_ID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Proyecto de integracion de datos**
Grupo conformado por:
*   Juanita Correa Amador
*   Santiago Arango Villegas
*   Juan Esteban Rave Ramirez

En este proyecto se contaba con una base de datos de generacion diaria y por hora de una hidroelectrica, esta base de datos contaba con 2959 registros y 31 columnas que indican la generacion por hora, total dia y fechas de cada año desde 2016 hasta 2025.

El proposito de esta integracion es clusterizar la base de datos para poder utilizarla con otros fines como realizar modelos predictivos y otros analisis que ayuden a mejorar la generacion de energia de la central para esto se utilizara el metodo K-medoids despues de realizar una limpieza, transformacion y extraccion de los datos esto con el fin de encontrar tendencias, patrones e informacion sobre el comportamiento y datos que contiene para poder clusterizar en base a eso.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Configuración inicial
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', 30)

In [3]:
nxl='/content/drive/MyDrive/Integración de datos y prospectiva/Copia de Consolidado_Generacion_Magallo_2016_2025.xlsx'
df = pd.read_excel(nxl)

**Limpieza BD**

In [4]:
# 1. Limpieza y preparación de datos
def preparar_datos(df):
    # Convertir fecha a formato datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True)

    # Reemplazar ceros por NaN (problema conocido)
    hour_columns = [str(h) for h in range(24)]
    df[hour_columns] = df[hour_columns].replace(0, np.nan)

    # Extraer componentes de fecha
    df['Año'] = df['Fecha'].dt.year
    df['Mes'] = df['Fecha'].dt.month
    df['Dia'] = df['Fecha'].dt.day
    df['Dia_semana'] = df['Fecha'].dt.dayofweek  # 0=Lunes, 6=Domingo

    # Ordenar por fecha
    df = df.sort_values('Fecha')

    return df

df = preparar_datos(df)
df

,Fecha,Recurso,0,1,2,3,4,5,6,7,8,9,10,11,12,...,14,15,16,17,18,19,20,21,22,23,TOTAL,Año,Mes,Dia,Dia_semana
2239,2016-12-21,MAGALLO,211.2,301.40,1447.60,1170.40,3410.00,4725.60,3425.40,NaN,NaN,1254.0,4081.0,2369.4,792.0,...,1771.0,2758.8,1918.4,NaN,NaN,836.0,3861.0,4232.8,4230.6,4224.0,47764.20,2016,12,21,2
2587,2016-12-22,MAGALLO,4239.4,4249.49,4249.49,4249.49,4252.85,4256.21,4262.93,4262.93,2679.46,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36702.25,2016,12,22,3
2949,2017-01-16,MAGALLO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,534.6,3163.6,2899.6,2899.6,2908.4,2910.6,2910.6,18227.00,2017,1,16,0
1900,2017-01-17,MAGALLO,2818.2,2774.20,2886.40,2886.40,2886.40,2886.40,2886.40,2884.20,2541.00,1353.0,1465.2,1089.0,2477.2,...,873.4,1568.6,2552.0,2321.0,2928.2,2926.0,2926.0,2930.4,3064.6,3066.8,59166.80,2017,1,17,1
1760,2017-01-18,MAGALLO,2939.2,2842.40,2842.40,2842.40,2842.40,2816.00,2758.80,2763.20,2857.80,2657.6,1973.4,2532.2,2358.4,...,2519.0,352.0,2840.2,2758.8,3011.8,2923.8,3009.6,2998.6,3082.2,2406.8,63866.00,2017,1,18,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2025-05-09,MAGALLO,5262.4,5258.00,5238.20,5240.40,5227.20,5264.60,5269.00,5277.80,5286.60,5222.8,5198.6,5304.2,5319.6,...,5315.2,5152.4,5150.2,5313.0,5295.4,5291.0,5302.0,5282.2,5277.8,5269.0,126330.60,2025,5,9,4
89,2025-05-10,MAGALLO,5275.6,5258.00,5247.00,5209.60,5240.40,5238.20,5233.80,5326.20,5330.60,5339.4,5337.2,5335.0,5332.8,...,5330.6,5335.0,5328.4,5330.6,5286.6,5286.6,5280.0,5280.0,5271.2,5269.0,127034.60,2025,5,10,5
85,2025-05-11,MAGALLO,5244.8,5280.00,5269.00,5244.80,5216.20,5271.20,5251.40,5299.80,5306.40,5295.4,5302.0,5304.2,5319.6,...,5330.6,5330.6,5332.8,5332.8,5330.6,5328.4,5324.0,5321.8,5324.0,5315.2,127201.80,2025,5,11,6
79,2025-05-12,MAGALLO,5313.0,5308.60,5317.40,5315.20,5315.20,5317.40,5317.40,5310.80,5302.00,5310.8,5308.6,5295.4,5293.2,...,5293.2,5280.0,5299.8,5306.4,5310.8,5304.2,5304.2,5304.2,5299.8,5295.4,127309.60,2025,5,12,0


**EDA Básico**

In [5]:
## 1. Análisis Descriptivo Interactivo
def analisis_descriptivo_plotly(df):
    hour_columns = [str(h) for h in range(24)]

    # Resumen estadístico
    print("Resumen estadístico por hora:")
    display(df[hour_columns].describe().transpose().style.background_gradient(cmap='YlOrRd'))

    # Boxplot + Scatter por año (como solicitaste)
    fig = make_subplots(rows=1, cols=2,
                       subplot_titles=('Distribución por Año',''),
                       shared_yaxes=True,
                       column_widths=[0.7, 0.3])

    # Boxplot
    for year in df['Año'].unique():
        fig.add_trace(go.Box(
            y=df[df['Año']==year]['TOTAL'],
            name=str(year),
            boxpoints='all', # no mostrar puntos individuales
            marker_color=px.colors.qualitative.Plotly[year % len(px.colors.qualitative.Plotly)]
        ), row=1, col=1)



    fig.update_layout(
        title='Generación Anual - Boxplot y Dispersión',
        yaxis_title='Generación Total Diaria',
        showlegend=False,
        height=600
    )
    fig.show()

    # Evolución temporal diaria
    fig = px.line(df, x='Fecha', y='TOTAL',
                 title='Evolución Temporal de la Generación Diaria',
                 hover_data={'Fecha': '|%Y-%m-%d', 'TOTAL': ':.2f'})
    fig.update_traces(line_width=2)
    fig.add_scatter(x=df['Fecha'], y=df['TOTAL'].rolling(30).mean(),
                   mode='lines', name='Media móvil (30 días)',
                   line=dict(color='red', width=2))
    fig.show()

analisis_descriptivo_plotly(df)

Resumen estadístico por hora:


,count,mean,std,min,25%,50%,75%,max
0,2944.000000,3394.395673,1316.920080,0.080000,2245.650000,3465.000000,4638.150000,7357.900000
1,2943.000000,3399.166442,1319.951426,0.090000,2231.900000,3465.000000,4650.800000,7343.800000
2,2942.000000,3409.962131,1317.971253,0.080000,2240.150000,3501.300000,4645.300000,7371.000000
3,2941.000000,3419.407929,1317.793852,0.080000,2261.600000,3500.200000,4670.600000,7360.300000
4,2942.000000,3425.997746,1315.282868,0.080000,2288.550000,3503.500000,4667.850000,7387.300000
5,2941.000000,3333.089565,1352.346783,0.080000,2131.800000,3359.400000,4615.600000,7405.800000
6,2937.000000,3158.899363,1435.266007,0.060000,1855.710000,3106.400000,4545.200000,7408.900000
7,2934.000000,3133.671530,1439.460473,0.060000,1832.600000,3076.700000,4529.800000,7427.800000
8,2926.000000,3122.747256,1431.826904,0.060000,1826.550000,3053.600000,4516.050000,7444.300000
9,2915.000000,3116.485804,1428.670700,0.070000,1820.500000,3044.800000,4496.800000,7470.400000


In [6]:
## 2. Heatmap Interactivo por Horas
def heatmap_interactivo(df):
    hour_columns = [str(h) for h in range(24)]

    # Heatmap diario
    fig = px.imshow(df[hour_columns].transpose(),
                   labels=dict(x="Fecha", y="Hora", color="Generación"),
                   x=df['Fecha'].dt.strftime('%Y-%m-%d'),
                   y=hour_columns,
                   color_continuous_scale='YlOrRd',
                   aspect='auto')
    fig.update_layout(title='Generación por Hora - Heatmap Diario',
                    xaxis_title='Fecha',
                    yaxis_title='Hora del día',
                    height=600)
    fig.show()

    # Heatmap promedio por hora y mes
    monthly_hour_means = df.groupby('Mes')[hour_columns].mean()
    fig = px.imshow(monthly_hour_means.transpose(),
                   labels=dict(x="Mes", y="Hora", color="Generación"),
                   color_continuous_scale='YlOrRd',
                   text_auto=".1f",
                   aspect='auto')
    fig.update_layout(title='Generación Promedio por Hora y Mes',
                    xaxis_title='Mes',
                    yaxis_title='Hora del día')
    fig.show()

heatmap_interactivo(df)

In [7]:
## 3. Análisis de Valores Atípicos Interactivo
def analisis_atipicos_plotly(df):
    hour_columns = [str(h) for h in range(24)]

    # Detección de atípicos usando método IQR
    outliers_data = []
    for hour in hour_columns:
        q1 = df[hour].quantile(0.25)
        q3 = df[hour].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        atipicos = df[(df[hour] < lower_bound) | (df[hour] > upper_bound)]
        if not atipicos.empty:
            for _, row in atipicos.iterrows():
                outliers_data.append({
                    'Fecha': row['Fecha'],
                    'Hora': hour,
                    'Generación': row[hour],
                    'Tipo': 'Bajo' if row[hour] < lower_bound else 'Alto'
                })

    if outliers_data:
        outliers_df = pd.DataFrame(outliers_data)
        fig = px.scatter(outliers_df, x='Fecha', y='Generación', color='Tipo',
                        facet_col='Hora', facet_col_wrap=6,
                        title='Valores Atípicos Detectados por Hora',
                        hover_data={'Fecha': '|%Y-%m-%d', 'Generación': ':.2f', 'Hora': True})
        fig.update_layout(height=800)
        fig.show()
    else:
        print("No se detectaron valores atípicos significativos.")

analisis_atipicos_plotly(df[df['Año']==2022])

In [8]:
## 4. Análisis por Día de la Semana
def analisis_dias_semana_plotly(df):
    # Asegurarnos de que la columna 'Dia_semana_nombre' existe
    if 'Dia_semana_nombre' not in df.columns:
        # Mapeo de números de día a nombres (0=Lunes, 6=Domingo)
        dias_map = {
            0: 'Monday',
            1: 'Tuesday',
            2: 'Wednesday',
            3: 'Thursday',
            4: 'Friday',
            5: 'Saturday',
            6: 'Sunday'
        }
        df['Dia_semana_nombre'] = df['Fecha'].dt.dayofweek.map(dias_map)

    # Orden de los días de la semana
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    # Calculamos el promedio por día de la semana
    weekday_avg = df.groupby('Dia_semana_nombre', observed=True)['TOTAL'].mean().reindex(weekday_order)

    # Gráfico de barras del promedio
    fig = px.bar(weekday_avg,
                labels={'value': 'Generación Promedio', 'index': 'Día de la semana'},
                title='Generación Promedio por Día de la Semana',
                text_auto='.2f')
    fig.update_layout(xaxis_title='Día de la semana',
                    yaxis_title='Generación Promedio Diaria')
    fig.show()

    # Boxplot + puntos por día de semana
    fig = go.Figure()

    for i, day in enumerate(weekday_order):
        day_data = df[df['Dia_semana_nombre'] == day]['TOTAL']

        # Boxplot
        fig.add_trace(go.Box(
            y=day_data,
            name=day,
            boxpoints=False,  # No mostrar puntos individuales en el boxplot
            marker_color=px.colors.qualitative.Plotly[i],
            showlegend=True
        ))

        # Puntos de dispersión
        fig.add_trace(go.Scatter(
            x=[day]*len(day_data),
            y=day_data,
            mode='markers',
            marker=dict(
                color=px.colors.qualitative.Plotly[i],
                size=6,
                opacity=0.5
            ),
            name=f'{day} (puntos)',
            showlegend=False,
            hovertext=df[df['Dia_semana_nombre'] == day]['Fecha'].dt.strftime('%Y-%m-%d')
        ))

    fig.update_layout(
        title='Distribución por Día de la Semana con Dispersión',
        yaxis_title='Generación Total Diaria',
        xaxis_title='Día de la semana',
        boxmode='group',
        height=600
    )
    fig.show()

# Llamamos a la función corregida
analisis_dias_semana_plotly(df)

In [9]:
## 5. Análisis Horario Detallado
def analisis_horario_detallado(df):
    hour_columns = [str(h) for h in range(24)]
    df_melted = df.melt(id_vars=['Fecha', 'Año'],
                       value_vars=hour_columns,
                       var_name='Hora',
                       value_name='Generación')

    # Convertir la columna Hora a numérico para ordenamiento correcto
    df_melted['Hora_num'] = df_melted['Hora'].astype(int)

    # Ordenar por hora numérica antes de agrupar
    df_melted = df_melted.sort_values('Hora_num')

    # 1. Evolución por hora a lo largo de los años
    fig = px.line(df_melted.groupby(['Año', 'Hora_num', 'Hora'])['Generación'].mean().reset_index(),
                 x='Hora',  # Mostramos la etiqueta de texto
                 y='Generación',
                 color='Año',
                 title='Evolución de la Generación Promedio por Hora (Orden Correcto)',
                 labels={'Generación': 'Generación Promedio'},
                 height=600,
                 category_orders={"Hora": hour_columns})  # Forzar orden correcto

    # Actualizar diseño
    fig.update_layout(
        xaxis_title='Hora del día',
        yaxis_title='Generación Promedio',
        xaxis=dict(
            type='category',  # Tratar como categorías pero mantener nuestro orden
            categoryorder='array',
            categoryarray=hour_columns  # Especificar el orden exacto
        )
    )
    fig.show()

    # 2. Boxplot por hora con puntos
    fig = go.Figure()

    # Obtenemos la paleta de colores una sola vez
    viridis_colors = px.colors.sequential.Viridis
    max_color_idx = len(viridis_colors) - 1  # Índice máximo disponible

    for hour in hour_columns:
        hour_data = df[hour].dropna()

        # Calculamos el índice del color de manera segura
        hour_num = int(hour)
        color_idx = int((hour_num / 23) * max_color_idx)
        color_idx = min(color_idx, max_color_idx)  # Aseguramos que no exceda el máximo

        color = viridis_colors[color_idx]

        # Boxplot
        fig.add_trace(go.Box(
            y=hour_data,
            name=f"Hora {hour}",
            boxpoints=False,
            marker_color=color,
            showlegend=True
        ))

        # Puntos de dispersión
        fig.add_trace(go.Scatter(
            x=[hour]*len(hour_data),
            y=hour_data,
            mode='markers',
            marker=dict(
                color=color,
                size=6,
                opacity=0.4,
                line=dict(width=0.5, color='DarkSlateGrey')
            ),
            name=f"Hora {hour} (puntos)",
            showlegend=False,
            hovertext=df.loc[hour_data.index, 'Fecha'].dt.strftime('%Y-%m-%d'),
            hoverinfo='y+text'
        ))

    fig.update_layout(
        title='Distribución por Hora del Día con Dispersión',
        yaxis_title='Generación (MWh)',
        xaxis_title='Hora del día',
        boxmode='group',
        height=700,
        hovermode='closest'
    )
    fig.show()

# Llamar a la función corregida
analisis_horario_detallado(df)

  Division del dataframe por años

In [10]:
df_por_año = {}
for año in df['Año'].unique():
    df_por_año[año] = df[df['Año'] == año].copy()

# Para verificar, puedes imprimir el tamaño de un dataframe de un año específico
# print(f"Registros para el año 2022: {len(df_por_año[2022])}")
# display(df_por_año[2022].head())

In [11]:
# @title  Método: Estimacion de Credibilidad
# Variables de Entrada: Dataset de observacion (XDo)
# Variables de Salida: Factor de credibilidad y Credibilidad(fc,Cr)
def credibilidad(XDo,XDe):

  # Se calcula el valor esperado de la varianza
  NDo=len(XDo); NDe=len(XDe)
  EPV=(NDo*np.var(XDo)+NDe*np.var(XDe))/(NDo+NDe)

  # Valor Hipotetico de la media (Es la media compartida)
  uh=(NDo*np.mean(XDo)+NDe*np.mean(XDe))/(NDo+NDe)

  # Varianza Hipotetica de la media (Varianza del total de los datos con respecto a la media compartida)
  VHM=((NDo*np.mean(XDo)**2+NDe*np.mean(XDe)**2)/(NDo+NDe))-uh**2

  #Factor de credibilidad
  fc=(EPV/VHM)

  #La credibilidad es
  Cr=NDo/(NDo+fc)

  return fc,Cr

Credibilidad para integracion

In [12]:
XDo = df[df['Año'].isin([2022, 2023, 2024, 2025])]

#Lista de años Restantes
años = [2016, 2017, 2018, 2019, 2020, 2021]
#Almacen de datos de credibilidad
lista_Credibilidad = []

#Ciclo para calcular la credibilidad de cada uno de los paises
for Año in años:
  XDe = df[df['Año'] == Año].copy()
  if not XDe.empty:
    fc, Cr = credibilidad(XDo['TOTAL'], XDe['TOTAL'])
    lista_Credibilidad.append({'Año': Año, 'Factor de credibilidad (fc)': fc, 'Credibilidad (Cr)': Cr})



credibility_df = pd.DataFrame(lista_Credibilidad)
display("Resultados de Factor y Credibilidad:")
display(credibility_df)

'Resultados de Factor y Credibilidad:'

,Año,Factor de credibilidad (fc),Credibilidad (Cr)
0,2016,497.871445,0.703802
1,2017,361.560286,0.765914
2,2018,91.103093,0.928496
3,2019,34.352353,0.971781
4,2020,53.479281,0.956749
5,2021,23.536591,0.980492


Modelo K-Medoids

In [41]:
hour_columns = [str(h) for h in range(24)]

# Impute NaN values in hourly columns with the column mean
for col in hour_columns:
    if df[col].isnull().any(): # Only try to fill if there are NaNs
        df[col] = df[col].fillna(df[col].mean())

feature_columns = hour_columns + ['TOTAL', 'Año', 'Mes', 'Dia', 'Dia_semana']
XD = np.array(df[feature_columns])
Yd = np.array(df[df['Año'].isin([2022, 2023, 2024, 2025])])

In [44]:
np.random.seed(42)
Xmin=np.min(XD,axis=0) #axis=0 buscar los minimos por cada columna
Xmax=np.max(XD,axis=0) #axis=0 buscar los máximos por cada columna

import random as rnd   #esta función genera números entre 0 y 1

XC=np.zeros((2, len(feature_columns)))  #Genera 4 grupos o clusters con el número correcto de variables

for j in range(2):
  XC[j,]=Xmin+(Xmax-Xmin)*rnd.random()
  XC[j,]=XD[j,]  #Tomamos los datos de los primeros 4 Indiviudos como medoids iniciales

fhat=np.zeros((len(XD),1))   #Esta variables nos determina el número de individuos por Bloque

for k in range(len(XD)):
  VP=np.exp(-0.5*(np.mean(((XC[:,]-XD[k,])/XC[:,])**2,axis=1)))
  nc=np.argmax(VP)
  fhat[k,]=int(nc)                  #Sabemos que dato pertenece a que cluster
  XC[nc,]=(XC[nc,]+XD[k,])/2   #Este es el K-medoids, a que cluster pertenece un dato

# Correct the loop to iterate through all 4 clusters
for j in range(2):
  npx=len(np.where(fhat[:,]==j)[0])
  print("El dato pertenece al cluster ",j," es ",npx)
  filas=np.where(fhat[:,]==j)[0]      #Filas de las personas Bloque tal

dfXC=pd.DataFrame(XC)
dfXC.columns=feature_columns
dfXC.index=[f'Cluster {i}' for i in range(len(XC))]
display(dfXC)

El dato pertenece al cluster  0  es  2
El dato pertenece al cluster  1  es  2957


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,TOTAL,Año,Mes,Dia,Dia_semana
Cluster 0,105.640000,150.745000,723.840000,585.240000,1705.040000,2362.840000,1712.730000,1566.865765,1561.403628,627.035000,2040.53500,1184.735000,396.030000,371.830000,885.530000,1379.430000,959.230000,1514.319714,1558.822888,418.040000,1930.54000,2116.440000,2115.340000,2112.040000,23882.950000,2017.5,10.000000,21.500000,2.50000
Cluster 1,5287.131943,5293.558788,5291.011979,5284.409605,5281.868201,5292.884876,5289.041847,5306.701952,5299.485531,5297.508513,5296.33113,5293.228201,5298.259521,5300.694054,5300.594954,5293.332207,5295.499324,5306.190011,5284.717325,5270.820569,5296.95196,5295.442006,5293.056037,5286.993096,127035.713628,2025.0,4.999878,12.003662,1.76378


In [45]:
df_integracion = df[df['Año'].isin([2018,2019,2020,2021])]
hour_columns2 = [str(h) for h in range(24)]

# Impute NaN values in hourly columns with the column mean
for col in hour_columns2:
    if df[col].isnull().any(): # Only try to fill if there are NaNs
        df[col] = df[col].fillna(df[col].mean())

feature_columns2 = hour_columns2 + ['TOTAL', 'Año', 'Mes', 'Dia', 'Dia_semana']
XD2 = np.array(df[feature_columns2])

In [46]:
XC2=np.copy(XC)

for k in range(len(XD2)):
  VP2=np.exp(-0.5*(np.mean(((XC2-XD2[k,])/XC2)**2,axis=1)))
  VP2Ing=np.exp(-0.5*((((XC2[:,2]-XD2[k,2])/XC2[:,2])**2)))

  if np.max(VP2Ing):
    nc2=np.argmax(VP2)
    print("El dato ",k,"pertenence al bloque ",nc2)

    #Actualizamos los clusters
    XC2[nc2,]=(XC2[nc2,]+XD2[k,])/2

#Los clusters originales son:
dfXC=pd.DataFrame(XC)
dfXC.columns=feature_columns
display(dfXC)

#Estos son clusters o bloques modificados
dfXC2=pd.DataFrame(XC2)
dfXC2.columns=feature_columns2
display(dfXC2)

#Evaluamos los cambios porcentuales de cada una de las variables en los clusters
print("Los cambios porcentuales en cada una de las variables son:\n")
pd.DataFrame(np.abs((XC-XC2)/XC))

El dato  0 pertenence al bloque  1
El dato  1 pertenence al bloque  1
El dato  2 pertenence al bloque  1
El dato  3 pertenence al bloque  1
El dato  4 pertenence al bloque  1
El dato  5 pertenence al bloque  1
El dato  6 pertenence al bloque  1
El dato  7 pertenence al bloque  1
El dato  8 pertenence al bloque  1
El dato  9 pertenence al bloque  1
El dato  10 pertenence al bloque  1
El dato  11 pertenence al bloque  1
El dato  12 pertenence al bloque  1
El dato  13 pertenence al bloque  1
El dato  14 pertenence al bloque  1
El dato  15 pertenence al bloque  1
El dato  16 pertenence al bloque  1
El dato  17 pertenence al bloque  1
El dato  18 pertenence al bloque  1
El dato  19 pertenence al bloque  1
El dato  20 pertenence al bloque  1
El dato  21 pertenence al bloque  1
El dato  22 pertenence al bloque  1
El dato  23 pertenence al bloque  1
El dato  24 pertenence al bloque  1
El dato  25 pertenence al bloque  1
El dato  26 pertenence al bloque  1
El dato  27 pertenence al bloque  1
El

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,TOTAL,Año,Mes,Dia,Dia_semana
0,105.640000,150.745000,723.840000,585.240000,1705.040000,2362.840000,1712.730000,1566.865765,1561.403628,627.035000,2040.53500,1184.735000,396.030000,371.830000,885.530000,1379.430000,959.230000,1514.319714,1558.822888,418.040000,1930.54000,2116.440000,2115.340000,2112.040000,23882.950000,2017.5,10.000000,21.500000,2.50000
1,5287.131943,5293.558788,5291.011979,5284.409605,5281.868201,5292.884876,5289.041847,5306.701952,5299.485531,5297.508513,5296.33113,5293.228201,5298.259521,5300.694054,5300.594954,5293.332207,5295.499324,5306.190011,5284.717325,5270.820569,5296.95196,5295.442006,5293.056037,5286.993096,127035.713628,2025.0,4.999878,12.003662,1.76378


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,TOTAL,Año,Mes,Dia,Dia_semana
0,52.860000,75.417500,361.960000,292.660000,852.560000,1181.460000,856.395000,783.462883,780.731814,313.552500,1020.30250,592.402500,198.045000,185.945000,442.795000,689.745000,479.645000,757.189857,779.441444,209.060000,965.31000,1058.260000,1057.710000,1056.060000,11942.325000,2018.25,9.000000,21.750000,2.75000
1,5287.131943,5293.558788,5291.011979,5284.409605,5281.868201,5292.884876,5289.041847,5306.701952,5299.485531,5297.508513,5296.33113,5293.228201,5298.259521,5300.694054,5300.594954,5293.332207,5295.499324,5306.190011,5284.717325,5270.820569,5296.95196,5295.442006,5293.056037,5286.993096,127035.713628,2025.00,4.999878,12.003662,1.76378


Los cambios porcentuales en cada una de las variables son:



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,0.499621,0.499701,0.499945,0.499932,0.499977,0.499983,0.499982,0.499981,0.499981,0.499944,0.499983,0.49997,0.499924,0.499919,0.499966,0.499978,0.499969,0.49998,0.499981,0.499904,0.499979,0.499981,0.499981,0.499981,0.499964,0.000372,0.1,0.011628,0.1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [47]:
from scipy.stats import skew, kurtosis
df_ob = df[df['Año'].isin([2022, 2023, 2024, 2025])]

for column in df_ob.columns:
    print(f"Información para la columna: {column}")
    if pd.api.types.is_numeric_dtype(df_ob[column]):
        print(f"  La media es: {np.mean(df_ob[column])}")
        print(f"  La varianza es: {np.var(df_ob[column])}")
        print(f"  El coeficiente de asimetría es: {skew(df_ob[column])}")
        print(f"  El coeficiente de curtosis es: {kurtosis(df_ob[column])}")
    else:
        print(f"  Value counts:\n{df_ob[column].value_counts()}")
    print("-" * 30)
for column in dfXC2.columns:
    print(f"Información para la columna: {column}")
    if pd.api.types.is_numeric_dtype(dfXC2[column]):
        print(f"  La media es: {np.mean(dfXC2[column])}")
        print(f"  La varianza es: {np.var(dfXC2[column])}")
        print(f"  El coeficiente de asimetría es: {skew(dfXC2[column])}")
        print(f"  El coeficiente de curtosis es: {kurtosis(dfXC2[column])}")
    else:
        print(f"  Value counts:\n{dfXC2[column].value_counts()}")
    print("-" * 30)

Información para la columna: Fecha
  Value counts:
Fecha
2025-05-13    1
2022-01-01    1
2022-01-02    1
2022-01-03    1
2022-01-04    1
             ..
2022-01-25    1
2022-01-19    1
2022-01-18    1
2022-01-17    1
2022-01-16    1
Name: count, Length: 1183, dtype: int64
------------------------------
Información para la columna: Recurso
  Value counts:
Recurso
MAGALLO    1183
Name: count, dtype: int64
------------------------------
Información para la columna: 0
  La media es: 3454.401841388649
  La varianza es: 1920516.424338587
  El coeficiente de asimetría es: -0.1093958242725196
  El coeficiente de curtosis es: -1.2714333019827304
------------------------------
Información para la columna: 1
  La media es: 3460.7095792155783
  La varianza es: 1929462.269590026
  El coeficiente de asimetría es: -0.1197450556512067
  El coeficiente de curtosis es: -1.28060894479671
------------------------------
Información para la columna: 2
  La media es: 3473.1127731665397
  La varianza es: 1914

In [48]:
from scipy.stats import skew, kurtosis

# Identify common numeric columns
common_numeric_columns = [col for col in df_ob.columns if pd.api.types.is_numeric_dtype(df_ob[col]) and col in dfXC2.columns]

results = []

for col in common_numeric_columns:
    # Calculate statistics for df_ob
    mean_ob = df_ob[col].mean()
    var_ob = df_ob[col].var(ddof=0) # Population variance
    skew_ob = skew(df_ob[col].dropna())
    kurt_ob = kurtosis(df_ob[col].dropna())

    # Calculate statistics for dfXC2
    mean_xc2 = dfXC2[col].mean()
    var_xc2 = dfXC2[col].var(ddof=0) # Population variance
    skew_xc2 = skew(dfXC2[col].dropna())
    kurt_xc2 = kurtosis(dfXC2[col].dropna())

    # Calculate percentage differences
    diff_mean = ((mean_xc2 - mean_ob) / mean_ob) * 100 if mean_ob != 0 else np.nan
    diff_var = ((var_xc2 - var_ob) / var_ob) * 100 if var_ob != 0 else np.nan
    diff_skew = ((skew_xc2 - skew_ob) / skew_ob) * 100 if skew_ob != 0 else np.nan
    diff_kurt = ((kurt_xc2 - kurt_ob) / kurt_ob) * 100 if kurt_ob != 0 else np.nan

    results.append({
        'Column': col,
        'Mean_Diff (%)': diff_mean,
        'Variance_Diff (%)': diff_var,
        'Skewness_Diff (%)': diff_skew,
        'Kurtosis_Diff (%)': diff_kurt
    })

diff_df = pd.DataFrame(results)
display(diff_df)

,Column,Mean_Diff (%),Variance_Diff (%),Skewness_Diff (%),Kurtosis_Diff (%)
0,0,-22.707430,256.643693,-100.0,57.302786
1,1,-22.429546,252.805532,-100.0,56.175701
2,2,-18.618076,217.292698,-100.0,56.349955
3,3,-19.981041,224.766537,-100.0,58.068116
4,4,-12.057882,157.107317,-100.0,57.695489
5,5,-4.605334,106.820238,-100.0,47.417455
6,6,-5.678828,116.491080,-100.0,40.849247
7,7,-6.092360,123.868411,-100.0,41.582311
8,8,-5.874934,125.711975,-100.0,45.223680
9,9,-13.312674,180.170908,-100.0,46.457044


##**Analisis de Resultados**
Este informe resume los hallazgos más significativos del análisis de los datos de generación eléctrica del archivo Consolidado_Generacion_Magallo_2016_2025.xlsx, cubriendo el período de 2016 a 2025.
Primero realizamos la visualización de la Generación Total Diaria la cual muestra la evolución a largo plazo, destacando fluctuaciones estacionales y posibles tendencias cíclicas o estructurales en la generación a lo largo del periodo despues realizamos unos diagramas de caja los cuales revelan que, si bien la mediana y el rango intercuartílico son generalmente consistentes, existen valores atípicos significativos en la generación total diaria para cada año, lo que sugiere eventos de generación inusualmente alta o baja.
El Heatmap Diario es crucial para identificar patrones horarios consistentes (horas pico y valle) y cómo estos cambian a lo largo del tiempo.El Heatmap Promedio por Hora y Mes indica que los perfiles de generación horaria varían significativamente entre meses, lo que apunta a una posible influencia de la demanda estacional o de las condiciones ambientales en la producción.
El análisis muestra la Generación Promedio por Día de la Semana, lo que permite identificar si existen diferencias significativas en la producción entre días laborables y fines de semana.Anomalías de 2022: La detección de valores atípicos en 2022 mediante el método IQR es fundamental para identificar periodos de generación anómala (inusualmente alta o baja) lo que se evidencio con esto es que en la mayoria de los años hubo dias anomalos donde la central estuvo sin generacion en algunas horas o incluso dias enteros donde se evidencia claramente riesgos operacionales para la central.
El gráfico de líneas de Evolución Promedio por Hora a lo Largo de los Años revela cómo el perfil de generación horario promedio ha cambiado con el tiempo, lo cual es vital para entender las tendencias a largo plazo en horas específicas.
El cálculo de credibilidad utilizando el período 2022-2025 como referencia ya que se evidencio en analisis anterirores como estos años eran los que contaban con menos valores atipicos a diferencia del resto de años y ademas se evidencion con el calculo de la credibilidad que esta aumenta significativamente a medida que los años son más recientes ademas de que se puede ver claramente que los años que mas se parecen a los observados son el eriodo del 2018 al 2021 principalmente 2019 y 2021
Para la intergracion mediante el proceso K-medoids Los valores faltantes (NaN) se imputaron con la media horaria para permitir el correcto funcionamiento del algoritmo K-Medoids. El clustering inicial reveló un fuerte desequilibrio en el tamaño de los grupos, con la mayoría de los datos concentrados en el Cluster 2 y el Cluster 1 , mientras que el Cluster 0 solo contenía 1 dato.
Se realizo la integracion de los datos desde 2018 a 2021 ya que todos tenian un valor de credibilidad de mas del 90% y se integraron 2958 datos de los cuales la mayoria se agruparon en los clusters 1 y 2 y mostrando cambios porcentuales bajos tanto entre los clusters como en sus estadisticos, expectuando el coeficiente de asimetria que mostro cambios porcentaluas para algunas categorias de mas del 100%.